In [ ]:
from IPython.display import display
from IPython.display import display
from rs_classes import annotation as annotation
from rs_classes import async_request_client as async_client
import _functions
import plotly.express as px
from sklearn.neighbors import LocalOutlierFactor

# #Initialize client
client = async_client.AsyncRequestClient("", "")
# Initialize the set_widgets list
url_input, bool_toggle, dropdown = _functions.create_input_widgets()
display(dropdown, url_input, bool_toggle)


### Limitations:
# 1. Only header fields for now
# 2. Not showing absent annotations for field (either not in schema or no BB)
# 3. Skipping if number of annotations is < than set up n_neighbors. Default is 5 annotations (n_neighbors). 
# (!) So if there is no group per 'slicer_field_id" with 6+ annotations -> no separate scattered chart is built for it


##########################################
token_input = ""
query = {
    "query": {
        "$and": [
            # {
            #     "queue": {
            #         "$in": [
            #             # "https://elis.rossum.ai/api/v1/queues/XXXXXXX",
            #             # "https://elis.rossum.ai/api/v1/queues/XXXXXXX"                                          
            #         ]
            #     }
            # },
            # {
            #     "field.sender_name.string": {
            #         "$eq": "Contoso Ltd"
            #     }
            # },
            {"status": {"$in": ["confirmed", "exported"]}}
        ]
    }
}

In [ ]:
field_id = ""
slicer_field_id = ""

################################

annotations_collection = await _functions.annotations_position_analysis(
    client, token_input, url_input, query, bool_toggle, dropdown, page_max=None
)

# df = output
df = _functions.annotations_position_post_processing(
    annotations_collection, field_id, slicer_field_id
)

df = df.dropna()

# Create scatter plot using Plotly Express
fig = px.scatter(
    df,
    x="center_x_percent",
    y="center_y_percent",
    facet_col="page",
    hover_data={
        "annotation_id": True,
        "center_x_percent": False,
        "center_y_percent": False,
    },
)

fig.update_layout(
    title="All Slicers",
    width=800,
    height=800,
)

fig.update_layout(
    xaxis=dict(range=[0, 100]),
    yaxis=dict(range=[100, 0])  # Reversing y-axis while maintaining range
)

fig.show()

# Get unique slicer values
unique_slicers = df["slicer"].unique()

# Plotting
for slicer_value in unique_slicers:
    # Filter the DataFrame based on the current slicer value
    sliced_data = df[df["slicer"] == slicer_value]
    
    #### n_neigbors
    n_neigbors = 1
    ### 

    contamination = 0.1
    outlier_labels = None

    if len(sliced_data) > n_neigbors:
        lof = LocalOutlierFactor(n_neighbors=n_neigbors)  # Adjust parameters as needed
        outlier_labels = lof.fit_predict(sliced_data[['center_x_percent','center_y_percent']])        

        # Create scatter plot using Plotly Express
        fig = px.scatter(
            sliced_data,
            x="center_x_percent",
            y="center_y_percent",
            color=outlier_labels,
            facet_col="page",
            hover_data={
                "annotation_id": True,
                "center_x_percent": False,
                "center_y_percent": False
            },
        )

        fig.update_layout(width=450, height=450, title="Empty" if slicer_value == '' else slicer_value)
        fig.update_layout(xaxis=dict(range=[0, 100]), yaxis=dict(range=[100, 0]))

        fig.show()

### To show original data set
# import pandas as pd
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

# display(df)